# AIの中間層の可視化

In [ ]:
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, Input

## MNISTデータの読み込み

embeddingsを取得するために、元データが必要。

In [ ]:
# MNISTデータの読み込み
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# データの正規化
train_images = train_images / 255.0
test_images = test_images / 255.0

In [ ]:
# 検証データを、上下左右反転させる
flipped_test_images = np.flip(test_images, axis=1)

# データの形状確認
print("Training data shape:", flipped_test_images.shape)

# いくつか表示
print("===================test data===============================")
num_rows, num_cols = 3, 10  # 行数と列数
fig, axes = plt.subplots(num_rows, num_cols, figsize=(1.5 * num_cols, 2 * num_rows))
for i in range(num_rows * num_cols):
    ax = axes[i // num_cols, i % num_cols]
    ax.imshow(test_images[i], cmap='gray')
    ax.set_title('Label: {}'.format(test_labels[i]))
plt.tight_layout()
plt.show()

print("===================flipped test data===============================")
num_rows, num_cols = 3, 10  # 行数と列数
fig, axes = plt.subplots(num_rows, num_cols, figsize=(1.5 * num_cols, 2 * num_rows))
for i in range(num_rows * num_cols):
    ax = axes[i // num_cols, i % num_cols]
    ax.imshow(flipped_test_images[i], cmap='gray')
    ax.set_title('Label: {}'.format(test_labels[i]))
plt.tight_layout()
plt.show()


## モデルの読み込み

In [ ]:
# モデルの読み込み
loaded_model = tf.keras.models.load_model('model.keras')  # または 'model.h5'
loaded_model.summary()

## 埋め込みembeddings (各層のベクトル)の取得
### 各層のembeddings取得用関数定義

In [ ]:
# Embeddingsを取得する関数
def get_embeddings_by_layer_name(model, input_data, layer_name):
    intermediate_layer_model = Model(inputs=model.input, outputs=model.get_layer(layer_name).output)
    embeddings = intermediate_layer_model.predict(input_data)
    return embeddings

### embeddingsの取得

In [ ]:
# 訓練データのembeddings取得
embeddings_train_dense_2 = get_embeddings_by_layer_name(loaded_model, train_images, 'dense_2')  # 第2のDense層の出力
## 検証データのembeddings取得
embeddings_test_dense_2 = get_embeddings_by_layer_name(loaded_model, test_images, 'dense_2')  # 第2のDense層の出力
## 反転させた検証データのembeddings取得
embeddings_flipped_test_dense_2 = get_embeddings_by_layer_name(loaded_model, flipped_test_images, 'dense_2')  # 第2のDense層の出力

## toorPIAによる中間層のembeddingsの可視化

In [ ]:
import toorpia.utils as tp

### 訓練データのembeddingsの解析

In [ ]:
%%time
# toorpiaによるembeddingsの可視化

common_options = {
#    'item_normalization': True,
#    'vector_normalization': True,
    'randomSeed': 0,
}

# 各fit_transform関数に共通のオプションを適用する
results_train_dense_2 = tp.fit_transform(embeddings_train_dense_2, working_dir='analysis_dense_2', **common_options)

### 検証データのembeddingsの解析

In [ ]:
%%time
# toorpiaによるembeddingsの可視化
results_test_dense_2 = tp.fit_transform(embeddings_test_dense_2, addplot=True, working_dir='analysis_dense_2', **common_options)

### 反転させた検証データのembeddingsの解析

In [ ]:
%%time
# toorpiaによるembeddingsの可視化
results_flipped_test_dense_2 = tp.fit_transform(embeddings_flipped_test_dense_2, addplot=True, working_dir='analysis_dense_2', **common_options)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Set up the figure and axes for two side-by-side plots
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(36, 12))  # Set total figure size

# Data and labels for the first plot (training)
x_train, y_train = results_train_dense_2[:, 0], results_train_dense_2[:, 1]

# Markers setup
markers = ['o', 's', '^', 'v', '<', '>', 'p', '*', 'h', 'D']

# First subplot (training data)
for i in sorted(range(10), reverse=False):
    idx = np.where(train_labels == i)
    ax1.scatter(x_train[idx], y_train[idx], c=[plt.cm.jet(i/9)], marker=markers[i], label=str(i), alpha=0.2, s=10)

ax1.set_xlim(-4, 4)
ax1.set_ylim(-4, 4)
ax1.set_aspect('equal')
ax1.legend()
ax1.set_title('Training Data')

# Data and labels for the second plot (test)
x_test, y_test = results_test_dense_2[:, 0], results_test_dense_2[:, 1]

# Base scatter for the test data, using training data as the base
ax2.scatter(x_train, y_train, label='base', c='lightgray', alpha=0.2, s=10)

# Second subplot (test data)
for i in sorted(range(10), reverse=False):
    idx = np.where(test_labels[0:len(x_test)] == i)
    ax2.scatter(x_test[idx], y_test[idx], c=[plt.cm.jet(i/9)], marker=markers[i], label=str(i), alpha=0.5, s=25)

ax2.set_xlim(-4, 4)
ax2.set_ylim(-4, 4)
ax2.set_aspect('equal')
ax2.legend()
ax2.set_title('Test Data')

# Data and labels for the second plot (test)
x_flipped_test, y_flipped_test = results_flipped_test_dense_2[:, 0], results_flipped_test_dense_2[:, 1]

# Base scatter for the test data, using training data as the base
ax3.scatter(x_train, y_train, label='base', c='lightgray', alpha=0.2, s=10)

# Second subplot (test data)
for i in sorted(range(10), reverse=False):
    idx = np.where(test_labels[0:len(x_test)] == i)
    ax3.scatter(x_flipped_test[idx], y_flipped_test[idx], c=[plt.cm.jet(i/9)], marker=markers[i], label=str(i), alpha=0.5, s=25)

ax3.set_xlim(-4, 4)
ax3.set_ylim(-4, 4)
ax3.set_aspect('equal')
ax3.legend()
ax3.set_title('Test Data')

# Show the combined plot
plt.show()